In [ ]:
%matplotlib inline


# 02. Simulate alpha waves

This example demonstrates how to simulate alpha waves using
HNN-core.



In [ ]:
# Authors: Mainak Jas <mainak.jas@telecom-paristech.fr>
#          Sam Neymotin <samnemo@gmail.com>

import os.path as op

Let us import hnn_core



In [ ]:
import hnn_core
from hnn_core import simulate_dipole, read_params, Network

Then we setup the directories and Neuron



In [ ]:
hnn_core_root = op.dirname(hnn_core.__file__)

Then we read the default parameters file



In [ ]:
params_fname = op.join(hnn_core_root, 'param', 'default.json')
params = read_params(params_fname)
print(params)

Update a few of the default parameters related to visualisation



In [ ]:
params.update({
    'dipole_scalefctr': 150000.0,
    'dipole_smooth_win': 0,
    'tstop': 310.0,
})

Now let's simulate the dipole and plot it. To excite the network, we add a
~10 Hz "bursty" drive starting at 50 ms and continuing to the end of the
simulation. Each burst consists of a pair (2) of spikes, spaced 10 ms apart.
The occurrence of each burst is jittered by a random, normally distributed
amount (20 ms standard deviation). We repeat the burst train 10 times, each
time with unique randomization. The drive is only connected to the distal
(dendritic) AMPA synapses on L2/3 and L5 pyramidal neurons.



In [ ]:
net = Network(params)

weights_ampa = {'L2_pyramidal': 5.4e-5, 'L5_pyramidal': 5.4e-5}
net.add_bursty_drive(
    'bursty', tstart=50., burst_rate=10, burst_std=20., numspikes=2,
    spike_isi=10, repeats=10, location='distal', weights_ampa=weights_ampa,
    seedcore=4)

dpl = simulate_dipole(net)

trial_idx = 0  # single trial simulated
dpl[trial_idx].plot()

We can confirm that what we simulate is indeed 10 Hz activity.



In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
import numpy as np
sfreq = 1000. / params['dt']
n_fft = 1024 * 8
freqs, _, psds = spectrogram(
    dpl[0].data['agg'], sfreq, window='hamming', nfft=n_fft,
    nperseg=n_fft, noverlap=0)
plt.figure()
plt.plot(freqs, np.mean(psds, axis=-1))
plt.xlim((0, 40))
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD')
plt.show()